In [116]:
import numpy as np
import time
import timeit
from timeit import default_timer as timer

class Backprop:
    
    # class members
    
    layerCount = 0
    shape = None
    weights = []
    
    #class methods
    
    def __init__(self, layerSize):
        
        self.layerCount = len(layerSize) - 1
        self.shape = layerSize
        
        # creating a list for input and output 
        
        self._layerInput = []
        self._layerOutput = []
        
        #creating weight arrays
        
        for (l1,l2) in zip(layerSize[:-1], layerSize[1:]):
            self.weights.append(np.random.normal(scale=1.5, size = (l2, l1+1)))
            
            
    def Run(self, input):
        
        lnCases = input.shape[0]
        
        # Clear out the previous intermediate value lists
        self._layerInput = []
        self._layerOutput = []
        
        # Run it!
        for index in range(self.layerCount):
            # Determine layer input
            if index == 0:
                layerInput = self.weights[0].dot(np.vstack([input.T, np.ones([1, lnCases])]))
            else:
                layerInput = self.weights[index].dot(np.vstack([self._layerOutput[-1], np.ones([1, lnCases])]))
                
            self._layerInput.append(layerInput)
            self._layerOutput.append(self.sgm(layerInput))
            
        return self._layerOutput[-1].T
    
    def TrainEpoch(self, input, target, trainingRate = 0.05):
    
        
        delta = []
        lnCases = input.shape[0]
       
        self.Run(input)
        
        # Calculating our deltas
        for index in reversed(range(self.layerCount)):
            if index == self.layerCount - 1:
                # Compare to the target values
                output_delta = self._layerOutput[index] - target.T
                error = np.sum(output_delta**2)
                delta.append(output_delta * self.sgm(self._layerInput[index], True))
            else:
                # Comparing to the following layer's delta
                delta_pullback = self.weights[index + 1].T.dot(delta[-1])
                delta.append(delta_pullback[:-1, :] * self.sgm(self._layerInput[index], True))
        
        
        # calculating weight deltas
        for index in range(self.layerCount):
            
            delta_index = self.layerCount - 1 - index
            
            if index == 0:
                layerOutput = np.vstack([input.T, np.ones([1, lnCases])])
            else:
                layerOutput = np.vstack([self._layerOutput[index - 1], np.ones([1, self._layerOutput[index - 1].shape[1]])])
            
            weightDelta = np.sum(\
                                 layerOutput[None,:,:].transpose(2, 0 ,1) * delta[delta_index][None,:,:].transpose(2, 1, 0)\
                                 , axis = 0)
            
                        
            self.weights[index] -= trainingRate * weightDelta
              
        return error
            
    def sgm(self, x, Derivative=False):
        if not Derivative:
            return 1 / (1+np.exp(-x))
        else:
            out = self.sgm(x)
            return out*(1-out)
            
if __name__ == "__main__":
        bpn = Backprop((2,2,2))
        print(bpn.shape)
        print(bpn.weights)
        
        lvInput = np.array([[-5.18, -5.35],
                             [-4.53, -2.32],
                             [-1.90, -1.80],
                             [-3.80, -2.29],
                             [-0.81, 0.46],
                             [1.14, -1.69],
                             [-0.79, 2.80],
                             [2.15, 0.97],
                             [4.24, 4.43],
                             [2.16, 3.76],
                             [5.93, 3.17],
                             [4.30, 4.97],
                             [6.73, 6.71],
                             [6.47, 7.45],
                             [10.38, 8.71],
                             [10.77, 8.42],
                            [-25.86, 16.78],
                             [-22.13, 16.41],
                             [-22.49, 17.52],
                             [-20.47, 16.70],
                             [-21.17, 19.82],
                             [-20.62, 20.93],
                             [-18.44, 19.99],
                             [-16.49, 21.96],
                             [-15.60, 24.03],
                             [-15.11, 25.53],
                             [-16.99, 25.88],
                             [-15.00, 26.42],
                             [-12.02, 28.35],
                             [-13.72, 28.11],
                             [-12.53, 27.80],
                             [-9.24,  31.88],
                             [14.29, -25.15],
                             [14.86, -23.98],
                             [17.81, -21.47],
                             [19.69, -21.54],
                             [20.57, -20.38],
                             [21.48, -20.41],
                             [19.01, -20.28],
                             [23.39, -17.17],
                             [23.07, -15.62],
                             [25.95, -14.36],
                             [24.72, -15.22],
                             [25.32, -15.81],
                             [25.53, -14.74],
                             [26.45, -12.21],
                             [30.94, -9.23],
                             [30.65, -8.30],
                            [-40.82, -38.45],
                            [-40.97, -40.46],
                            [-38.55, -37.16],
                            [-38.34, -38.00],
                            [-36.02, -36.73],
                            [-33.14, -35.97],
                            [-32.51, -34.50],
                            [-32.84, -34.33],
                            [-34.92, -35.84],
                            [-34.46, -37.86],
                            [-36.42, -38.53],
                            [-39.26, -36.34],
                            [-38.35, -40.79],
                            [-41.45, -40.80],
                            [-41.52, -40.21],
                            [-41.39, -41.88]])
        
        lvTarget = np.array([[0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                             [0.05, 0.05],
                            [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                             [0.05, 0.95],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05],
                            [0.95, 0.05]])
        
        lnMax = 500000
        lnErr = 0.001
        start = timer()
        for i in range(lnMax+1):
            
            err = bpn.TrainEpoch(lvInput, lvTarget)
            if i % 5000 == 0:
                print("Iterations {0}\tError: {1:0.6f}".format(i, err))
            if err <= lnErr:
                print("Minimum error reached at Iteration: {0}".format(i))
                break
        end = timer()
        print("time taken in seconds")  
        print(end-start)
        lvOutput = bpn.Run(lvInput)
        
        print("input : {0}\Output: {1}".format(lvInput, lvOutput))
       

(2, 2, 2)
[array([[ 0.82265009, -1.13033317,  2.53111394],
       [ 0.56543801, -2.08807359,  0.02109704]]), array([[ 0.51311718,  2.59238338,  0.95624835],
       [ 1.80176428,  1.32751936,  0.98736173]])]
Iterations 0	Error: 58.773463
Iterations 5000	Error: 12.957425
Iterations 10000	Error: 12.935319
Iterations 15000	Error: 12.962746
Iterations 20000	Error: 12.963684


KeyboardInterrupt: 